In [ ]:
import pandas as pd
import requests
from urllib.parse import urlencode
import csv

def fetch_single_spectrum():
    base_url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
    query = "SELECT * FROM spectra"
    params = {"query": query, "format": "csv"}
    url = f"{base_url}?{urlencode(params)}"

    try:
        df = pd.read_csv(url)
        print(df.head())  # Debugging: Print metadata preview

        base_data_url = "https://exoplanetarchive.ipac.caltech.edu/workspace/TMP_0S5fAs_13661/atmospheres/tab1/data/"
        csv_filename = "MyData.csv"

        with open(csv_filename, mode="w", newline="") as file:
            writer = csv.writer(file)
            header_written = False  # Track if the header has been written

            for spec_path in df['spec_path']:
                full_url = f"{base_data_url}{spec_path}"
                print(f"Downloading {full_url}")  # Debugging: Print the full URL

                response = requests.get(full_url)

                if response.status_code == 200:
                    content = response.text.split("\n")

                    for line in content:
                      if line.startswith("\\PL_NAME"):
                        planet_name = line.split("=")[1].strip()
                        break
                    table_lines = [line.strip() for line in content if not "\\" in line]
                    print(table_lines)

                    # Find the **first real data row** by ignoring headers, types, and units
                    data_start_index = -1
                    for i, line in enumerate(table_lines):
                        if "microns" in line or "Jupiter radii" in line or "days" in line:  # This row is just units, skip it
                            continue
                        if "double" in line or "char" in line or "null" in line:  # Data types and placeholders, skip them
                            continue
                        data_start_index = i
                        break  # Found first valid data row

                    if data_start_index == -1:
                        raise ValueError("Error: No valid data rows found!")

                    # Extract headers (before the first data row)
                    header = ["PL_NAME"] + [col.strip() for col in table_lines[data_start_index].split("|") if col.strip()]

                    # Write header only once
                    if not header_written:
                        writer.writerow(header)
                        header_written = True

                    for j in range(data_start_index + 4, len(table_lines)):
                        temp_data = [col.strip() for col in table_lines[j].split(" ") if col.strip()]
                        data = []
                        i = 0
                        while i < len(temp_data):
                            if "et" in temp_data[i]:
                                # Merge author name parts
                                data.append(temp_data[i-1] + " " + temp_data[i] + " " + temp_data[i+1] + " " + temp_data[i+2])
                                data.remove(temp_data[i-1])
                                i += 2
                            elif temp_data[i].startswith("http"):
                                data.append(temp_data[i])
                            else:
                                data.append(temp_data[i])
                            i += 1

                        print(data)

                        # Ensure the data row has the same length as the header
                        if len(data) != len(header):
                          print(f"Warning: Data row length ({len(data)}) and header length ({len(header)}) do not match.")

                          if len(data) > len(header):
                              data = data[:len(header)]  # Trim excess values
                          else:
                              data += [""] * (len(header) - len(data))  # Pad with empty strings

                        writer.writerow([planet_name] + data)  # Writing all rows properly
                        print("Data row appended successfully.")

                else:
                    print(f"Failed to download. HTTP Status: {response.status_code}")

        print(f"Data successfully written to {csv_filename}")

    except Exception as e:
        print(f"Error fetching data: {e}")

# Example usage
fetch_single_spectrum()

Streaming output truncated to the last 5000 lines.
Data row appended successfully.
['2.06425', '0.00098', '1.18116', '0.03745', '-0.03745', '0', 'Louie et al. 2025', 'https://ui.adsabs.harvard.edu/abs/2025AJ....169...86L/abstract', '0.10868', '0.00171', '-0.00174', '1', 'Calculated', '/docs/atmospheres/atmospheres_calc.html', '1.5758', '0.2025', '-0.2025', '0', 'Calculated', '/docs/atmospheres/atmospheres_calc.html', '2460024.197422', '0.000019', '-0.000019', '0', '1.4900', '0.1900', '-0.1900', 'Stassun et al. 2017', 'https://ui.adsabs.harvard.edu/abs/2017AJ....153..136S/abstract']
Data row appended successfully.
['2.06524', '0.00098', '1.16585', '0.03805', '-0.03805', '0', 'Louie et al. 2025', 'https://ui.adsabs.harvard.edu/abs/2025AJ....169...86L/abstract', '0.10797', '0.00175', '-0.00178', '1', 'Calculated', '/docs/atmospheres/atmospheres_calc.html', '1.5656', '0.2013', '-0.2013', '0', 'Calculated', '/docs/atmospheres/atmospheres_calc.html', '2460024.197422', '0.000019', '-0.000019'